In [1]:
!python -m pip install torch torchvision --index-url https://download.pytorch.org/whl/cu126

Looking in indexes: https://download.pytorch.org/whl/cu126
INFO: pip is looking at multiple versions of networkx to determine which version is compatible with other requirements. This could take a while.
  Using cached https://download.pytorch.org/whl/networkx-3.3-py3-none-any.whl.metadata (5.1 kB)
  Using cached https://download.pytorch.org/whl/mpmath-1.3.0-py3-none-any.whl (536 kB)
   ---------------------------------------- 0.0/2.6 GB ? eta -:--:--
   ---------------------------------------- 0.0/2.6 GB 5.6 MB/s eta 0:07:43
   ---------------------------------------- 0.0/2.6 GB 5.9 MB/s eta 0:07:17
   ---------------------------------------- 0.0/2.6 GB 5.6 MB/s eta 0:07:42
   ---------------------------------------- 0.0/2.6 GB 5.9 MB/s eta 0:07:20
   ---------------------------------------- 0.0/2.6 GB 5.6 MB/s eta 0:07:42
   ---------------------------------------- 0.0/2.6 GB 6.0 MB/s eta 0:07:12
   ---------------------------------------- 0.0/2.6 GB 6.0 MB/s eta 0:07:10
   ---------

In [ ]:
import zipfile
from PIL import Image
import torch
from torchvision import transforms
from io import BytesIO
from torch.utils.data import DataLoader, TensorDataset
import numpy as np


device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("GPU is", "available" if device else "NOT AVAILABLE")

# Caminho para o arquivo ZIP
zip_path1 = 'bird.zip'
zip_path2 = 'not-bird.zip'

# Transforms para redimensionar e converter para tensor
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),  # Converte para [C x H x W]
])

# Lista para armazenar os tensores
image_in_tensors = []
image_out_tensors = []


# Abre o ZIP e processa as imagens diretamente
def loadImages(zip_path,label,max):
    count = 0
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        for file_name in zip_ref.namelist():
            if file_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                with zip_ref.open(file_name) as file:
                    image = Image.open(BytesIO(file.read())).convert('RGB')
                    tensor = transform(image)  # Shape: [3, 256, 256]
                    #print(tensor.shape)
                    image_in_tensors.append(tensor)
                    image_out_tensors.append([label])
                count+=1
                if count%1000 == 0:
                    print(count)
                    
                if count >= max:
                    return

# Empilha os tensores em um batch
loadImages(zip_path1,1,2000)
loadImages(zip_path2,0,2000)

print(f'Total de imagens carregadas: {len(image_in_tensors)}')

t_x = torch.stack(image_in_tensors)
t_y = torch.tensor(image_out_tensors,dtype=torch.float32)

shuffler = np.random.permutation(len(t_x))

x_shuffled = t_x[shuffler]
y_shuffled = t_y[shuffler]

t_xt = x_shuffled.to(device)
t_yt = y_shuffled.to(device)

dataset = TensorDataset(t_xt, t_yt)
#batch_tensor = torch.stack(image_in_tensors)  # Shape: [N, 3, 256, 256]


print(f'Shape do batch: {t_xt.shape}')
print(f'Shape dos rótulos: {t_yt.shape}')



GPU is available
1000
2000
1000
2000
Total de imagens carregadas: 4000
Shape do batch: torch.Size([4000, 3, 32, 32])
Shape dos rótulos: torch.Size([4000, 1])


In [6]:
import torch;
import torch.nn as nn

class RedeCnnBirdNotBird(nn.Module):
    def __init__(self):
        super(RedeCnnBirdNotBird, self).__init__()

        self.conv1 = nn.Conv2d(3, 6, 3, stride=1)
        self.conv2 = nn.Conv2d(6, 12, 3, stride=1)
        self.conv3 = nn.Conv2d(12, 24, 5, stride=1)
        self.poll1 = nn.MaxPool2d(2,2)
        self.poll2 = nn.MaxPool2d(2,2)
        
        self.linear1 = nn.Linear(864,256)
        self.linear2 = nn.Linear(256,1)
        #self.linear3 = nn.Linear(164,164)
        #self.linear4 = nn.Linear(512,1)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.poll1(x)
        x = self.poll2(x)
        
        #print("x poll1: ",x.size())
        
        x = torch.flatten(x, start_dim=1)
        
        #print("x flatten: ",x.size())
        
        x = torch.relu(self.linear1(x))
        x = torch.relu(self.linear2(x))
        
        #x = torch.relu(self.linear1(x))
        #x = torch.relu(self.linear2(x))
        #x = torch.tanh(self.linear3(x))
        #x = torch.relu(self.linear4(x))
        return x

In [15]:
import time

def train(cnn,dataset, epochs=10):
    opt = torch.optim.Adam(cnn.parameters(),lr=0.0001)#0.00001  #0.0000001

    train_loader = torch.utils.data.DataLoader(
        dataset, batch_size=64, shuffle=True
    )

    for epoch in range(epochs):
        totalloss = 0
        batch = 0
        start_time = time.time()
        opt.zero_grad()

        for inputs, targets in train_loader:
            
            #inputs = x[:, :insize]    
            #targets = x[:, insize:] 
            #x = x.to(device) # GPU
            x_hat = cnn(inputs)
            #print(inputs.size()," ",targets.size()," ",x_hat.size())
            loss = ((targets - x_hat)**2).sum()

            loss.backward(retain_graph=True)
            totalloss+=loss
            batch+=1
 
            opt.step()
            opt.zero_grad()
            #print("step: ")

        end_time = time.time()
        print(epoch," Total Loss: ",(totalloss/len(dataset))," time ",(end_time-start_time))
        #if epoch%250==0:
        #    torch.save(cnn, "backup_training_gam_v02.pth")
    return cnn


In [16]:
cnn = RedeCnnBirdNotBird().to(device)

cnn = train(cnn, dataset,epochs=100)





0  Total Loss:  tensor(0.2916, device='cuda:0', grad_fn=<DivBackward0>)  time  0.4323604106903076
1  Total Loss:  tensor(0.2292, device='cuda:0', grad_fn=<DivBackward0>)  time  0.2624471187591553
2  Total Loss:  tensor(0.2176, device='cuda:0', grad_fn=<DivBackward0>)  time  0.25267958641052246
3  Total Loss:  tensor(0.2052, device='cuda:0', grad_fn=<DivBackward0>)  time  0.25929951667785645
4  Total Loss:  tensor(0.1952, device='cuda:0', grad_fn=<DivBackward0>)  time  0.25402116775512695
5  Total Loss:  tensor(0.1856, device='cuda:0', grad_fn=<DivBackward0>)  time  0.2893669605255127
6  Total Loss:  tensor(0.1802, device='cuda:0', grad_fn=<DivBackward0>)  time  0.25309300422668457
7  Total Loss:  tensor(0.1758, device='cuda:0', grad_fn=<DivBackward0>)  time  0.24725580215454102
8  Total Loss:  tensor(0.1756, device='cuda:0', grad_fn=<DivBackward0>)  time  0.25619029998779297
9  Total Loss:  tensor(0.1707, device='cuda:0', grad_fn=<DivBackward0>)  time  0.2655973434448242
10  Total Loss

In [18]:

tabela_verdade = [
    [0, 0],  # classe 0
    [0, 0]   # classe 1
]

train_loader = torch.utils.data.DataLoader(
    dataset, batch_size=1, shuffle=True
)


for inputs, targets in train_loader:
    x_hat = cnn(inputs)
    classe = 0
    if x_hat[0] > 0.5:
        classe = 1
        
    #print(targets[0][0])
    tabela_verdade[classe][int(targets[0][0])]+=1
    
print(tabela_verdade)

print('   B','   NB')
print('PB ',tabela_verdade[1][1],' ',tabela_verdade[1][0])
print('PNB ',tabela_verdade[0][1],' ',tabela_verdade[0][0])

VP = tabela_verdade[1][1]
FP = tabela_verdade[1][0]
FN = tabela_verdade[0][1]
VN = tabela_verdade[0][0]
Total = VP+FP+FN+VN
ACC = (VP+VN)/Total
PRE = VP/(VP+FP)
REV = VP/(VP+FN)
F1 = (PRE*REV/(PRE+REV))*2

print("ACC ",ACC)
print("PRE ",PRE)
print("REV ",REV)
print("F1 ",F1)

[[1992, 20], [8, 1980]]
   B    NB
PB  1980   8
PNB  20   1992
ACC  0.993
PRE  0.9959758551307847
REV  0.99
F1  0.9929789368104314
